In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import random

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Check if GPU is available
# Check for CUDA (NVIDIA GPU) or MPS (Apple Silicon) availability
if torch.cuda.is_available():
    device = torch.device("cuda:0")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cpu


In [7]:
# Renommer les images capturées

# for action in ['LEFT', 'RIGHT', 'FIRE', 'ENTER'] :

  #  dossier_cible = f"Echantillons/{action}"  # Dossier dont renommer les images

   # images = os.listdir(dossier_cible)

   # for i, image in enumerate(images):
    #    os.rename(os.path.join(dossier_cible, image), os.path.join(dossier_cible, f"{action}_{i}.jpg"))

# print("Images renommées !")

FileExistsError: [WinError 183] Impossible de créer un fichier déjà existant: 'Echantillons/LEFT\\LEFT_10.jpg' -> 'Echantillons/LEFT\\LEFT_2.jpg'

In [3]:
# Définir les transformations des images (redimensionnement et normalisation)
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Redimensionne les images
    transforms.ToTensor()  # Convertit en tenseur PyTorch
])

# Charger les images depuis les dossiers
dataset = datasets.ImageFolder(root="Echantillons", transform=transform, is_valid_file=lambda x: x.endswith(('.jpg')))

# Vérifier les classes détectées
print("Classes détectées :", dataset.classes)

# Créer un DataLoader pour gérer le batching
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Calculer la moyenne et l'écart type du jeu de données
mean = torch.zeros(3)
std = torch.zeros(3)
num_samples = 0

for images, _ in data_loader:
    batch_samples = images.size(0)  # Nombre d'images dans le batch
    images = images.view(batch_samples, 3, -1)  # Réorganiser pour chaque canal
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    num_samples += batch_samples

mean /= num_samples
std /= num_samples

print("Mean:", mean)
print("Std:", std)

Classes détectées : ['ENTER', 'FIRE', 'LEFT', 'RIGHT']
Mean: tensor([0.5242, 0.5160, 0.4967])
Std: tensor([0.1961, 0.1897, 0.2083])


In [17]:
## Classes :

# 0 = 'ENTER'
# 1 = 'FIRE'
# 2 = 'LEFT'
# 3 = 'RIGHT'

In [5]:
len(dataset)

5631

In [5]:
# Recharger les données avec cette fois une normalisation en plus :

# Nouvelle transformation avec normalisation personnalisée
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

# Recharger le dataset avec la nouvelle transformation
dataset = datasets.ImageFolder(root="Echantillons", transform=transform)

# Séparer en train et test
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Création des DataLoaders
trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Dataloader mis à jour avec normalisation personnalisée !")

Dataloader mis à jour avec normalisation personnalisée !


In [7]:
# Fonction pour afficher une image
def imshow(img):
    # Dé-normalisation
    img = img.clone().detach()  
    img = img.numpy().transpose((1, 2, 0))  

    img = img * std + mean  # Appliquer la dé-normalisation
    img = np.clip(img, 0, 1)  

    plt.figure(figsize=(5, 5))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

# Récupérer un batch d'images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# Afficher une seule image du batch
imshow(images[0])

# Afficher son label
print('Label:', dataset.classes[labels[0]])

TypeError: unsupported operand type(s) for *: 'numpy.ndarray' and 'Tensor'